In [1]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions')
sys.path
import nltk
nltk.data.path = ['/vol/medic02/users/ag6516/nltk_data']

In [2]:
%matplotlib inline

import os
import csv
import json
import pickle
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn.decomposition import PCA
import numpy as np
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import random
from random import randint
random.seed(123)
np.random.seed(123)

import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

import torch

# Set THEANO_FLAGS='device=cuda0,floatX=float32' to run notebook on gpu
import lasagne
from lasagne.utils import floatX
import theano
import theano.tensor as T
from collections import Counter
from lasagne.utils import floatX

import googlenet

from read_data import read_csv_into_df
from vis_utils import kmeans_clustering, plot_pca, visualise_word_clusters2, kmeans_silhouette
from vis_utils import save_clusters_to_json, load_clusters_from_json

ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 164, in <module>
    use(config.device)
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 151, in use
    init_dev(device)
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 66, in init_dev
    avail = dnn.dnn_available(name)
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/dnn.py", line 174, in dnn_available
    if not dnn_present():
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/dnn.py", line 157, in dnn_present
    dnn_present.avail, dnn_present.msg = _dnn_check_version()
  File "/vol/medic02

In [3]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')
df = pd.read_csv(dir + 'cleaned_reports_edited.csv')

In [7]:
reports = []
reports_tok = []
labels = []
words = []
report_lengths = []
num_sentences = []

for i, row in df.iterrows():
    report = row['Report text'].decode('utf-8').lower()
    num_sentences.append(report.count('.'))
    # Remove markdown
    for char in ['\n', '\b']:
        report = report.replace(char, '')
    
    # Replace unnecessary punctuation 
    for char in ['~', '"']:
        report = report.replace(char, '')
        
    for char in ['!', '?', ';', ':', '.']:
        report = report.replace(char, ' . ')
    
    for char in ['(', ')', ',', '/', '\\']:
        report = report.replace(char, ' , ')
        
    # filter out 'comparison' + dates
    #report = re.sub('( compar.*?\d{4})', '', report)
    report = re.sub('(compar.*?\d{4})', '', report)
    report = re.sub('(compar.*?\d{2})', '', report)
    report = re.sub('(xr knee both)', '', report)
    report = re.sub('(xr knee)', '', report)
    report = re.sub('(previous.*comparison)', '', report)
    report = re.sub('( please.*?\.)', '', report)

    # Tokenize
    report_tok = word_tokenize(report)
    report_length = len(report_tok)
    
    # Remove stopwords
    filtered_report_tok = [word for word in report_tok if word not in stopwords.words('english')]
    filtered_report = ' '.join(filtered_report_tok)
    
    reports.append(filtered_report)
    reports_tok.append(filtered_report_tok)
    labels.append(row['Accession'])
    [words.append(word) for word in filtered_report_tok]
    report_lengths.append(report_length)

In [29]:
print 'Vocab length: ', len(sorted(set(words)))
fdist_all = nltk.FreqDist(words)
freqs = [freq for word, freq in fdist_all.most_common() if word not in ['.', ',']]
print 'Avg no. of appearances: ', np.mean(freqs)
print 'Max: ', fdist_all.most_common(2)
print 'STD: ', np.std(freqs)
#print fdist_all.hapaxes()
print 'Number of reports: ', len(reports)
print 'Average length of report: ', np.mean(report_lengths)
print 'STD: ', np.std(report_lengths)
print ''
print 'Average number of sentences: ', np.mean(num_sentences)
print 'STD: ', np.std(num_sentences)
print ''
df['Cleaned report'] = reports
print df.count()

Vocab length:  1735
Avg no. of appearances:  35.4039238315
Max:  [(u'.', 12474), (u'joint', 3749)]
STD:  170.997692276
Number of reports:  3561
Average length of report:  30.5237292895
STD:  18.2751155872

Average number of sentences:  2.70176916596
STD:  1.96696094788

Unnamed: 0          3561
Accession           3561
Clinical history    2221
Comment             2390
Report text         3561
Cleaned report      3561
dtype: int64


In [11]:
patient_ids = []
for folder in os.listdir(dir+'data/Images'):
    #os.path.exists(self.labelfilename)
    patient_ids.append(str(folder))

In [12]:
print len(patient_ids)

330


In [13]:
sample_images = []
views = {}
for pid in patient_ids:
    for image in os.listdir(dir + 'data/Images/' + str(pid)):
        if image.endswith('.jpg'):
            item = {}
            imid = os.path.splitext(image)[0] 
            item['patient id'] = pid
            item['impath'] = pid + '/' + image
            if pid in labels:
                item['report'] = reports[labels.index(pid)]
                item['tokens'] = reports_tok[labels.index(pid)]
                if '_' in image:             
                    leg = imid.split('_')[-1]
                    view = imid.split('_')[-2]
                
                    item['imid'] = imid.split('_')[0]
                    item['leg'] = leg
                    item['view'] = view
                    if view not in views.keys():
                        views[view] = 0
                    views[view] = views[view] + 1
                else:
                    item['imid'] = imid
            else: continue

            sample_images.append(item)
print views

{'HBL': 12, 'WBAP': 34, 'L': 90, 'AP': 69, 'S': 33, 'WBL': 17}


## Reduce vocab

In [25]:
allwords = Counter()
pids = set()
for item in sample_images:
    pid = item['patient id']
    if pid not in pids:
        tokens = item['tokens']
        allwords.update(tokens)

    pids.add(pid) 

vocab = [k for k, v in allwords.items() if v >= 10]

len(vocab)

105

In [30]:
for item in sample_images:
    old_tokens = item['tokens']
    item['tokens'] = [word for word in old_tokens if word in vocab]

In [31]:
train1, test = model_selection.train_test_split(sample_images, train_size=0.8, random_state=42)
train, val = model_selection.train_test_split(train1, train_size=0.8, random_state=42)

for item in train:
    item['split'] = 'train'

for item in val:
    item['split'] = 'val'

for item in test:
    item['split'] = 'test'

In [32]:
sample_images = train+test+val
print 'Train: ', len(train)
print 'Val: ', len(val)
print 'Test: ', len(test)
print 'Total', len(sample_images)

Train:  916
Val:  229
Test:  287
Total 1432


In [33]:
pickle.dump(sample_images, open('sample_exams_train_test_split.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)